In [1]:
import json
import codecs
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
import numpy as np

In [9]:
def compoundColumnsNamesList(input_json, node):

# copying  top level dataset with known top level parent node 'dates' into dataframe top_level_dataset

    top_level_dataset = json_normalize(input_json[node])

# saving names of columns of top_level_dataset into tld_columns_list 
    tld_columns_list = []
    for elem in top_level_dataset.columns.values:
        tld_columns_list.append(elem)
# print(tld_columns_list[:4])

# copying  2nd level dataset under parent node 'sessions' into dataframe sessions_dataset
    sessions_dataset = json_normalize(data = input_json[node], 
                               record_path = 'sessions')

# saving names of columns of sessions_dataset into sessions_columns_list 
    sessions_columns_list = []
    for elem in sessions_dataset.columns.values:
        sessions_columns_list.append(elem)
# print(sessions_columns_list)

# copying  3nd level dataset under parent node 'activities' into dataframe activities_dataset
    activities_dataset = json_normalize(data = input_json[node], 
                                    record_path = ['sessions','activities'])

# saving names of columns of activities_dataset into activities_columns_list 
    activities_columns_list = []
    for elem in activities_dataset.columns.values:
        activities_columns_list.append(elem)
# print(activities_columns_list)

# joining names of  top level colemn (excluding node name) -->
# with complex names of 2nd level columns (top level node name and names of -->
# columns under it) 
    columns_list = tld_columns_list[:4]
    param = [tld_columns_list[4]]
    for item in sessions_columns_list[1:]:
        param.append(item)
        columns_list.append(param)
        param = [tld_columns_list[4]]
    
# joining names of  top level colemn (excluding node name) -->
# with complex names of 2nd level columns (top level node name and names of -->
# columns under it) 
#columns_list = tld_columns_list[:4]
    param = [tld_columns_list[4]]
    param.append(sessions_columns_list[0])
    for item in activities_columns_list[1:]:
        param.append(item)
        columns_list.append(param)
        param = [tld_columns_list[4]]
        param.append(sessions_columns_list[0])

    return columns_list

In [10]:
def json2dataFrame(ID, input_json, node, path, columnsList):
    

# parsing json into dataFrame output_dataset
    output_dataset = json_normalize(data = input_json[node], 
                                    record_path = path,
                                    meta = columnsList)
#print(output_dataset)

#output_dataset.head(3)

    activities_dataset = json_normalize(data = input_json[node], 
                                        record_path = path[0:2]) 
                              

    output_dataset = pd.concat([output_dataset, 
                              activities_dataset.repeatActivityTimes], 
                              axis = 1)
    
#output_dataset.head(3)



    output_dataset['Client ID'] = ID

#output_dataset.head(3)

    for i in range(output_dataset.shape[0]):
        output_dataset.iat[i,6] = (output_dataset.iat[i,6])['PAGEVIEW']

#output_dataset.head(3)



    output_dataset.rename({output_dataset.columns.values[6] : output_dataset.columns.values[6]+' - PAGEVIEW'},
                           axis = 'columns', inplace = True)

    return output_dataset

In [14]:
clientsIDlist = pd.read_excel(io = 'ClientID.xlsx', sheet_name = 'list1',
                             index_col = [0])

In [15]:
z = list(clientsIDlist)

In [16]:
z

['client ID']

In [43]:
rowsNum = clientsIDlist.count(0)

In [45]:
help(clientsIDlist.count)

Help on method count in module pandas.core.frame:

count(axis=0, level=None, numeric_only=False) method of pandas.core.frame.DataFrame instance
    Return Series with number of non-NA/null observations over requested
    axis. Works with non-floating point data as well (detects NaN and None)
    
    Parameters
    ----------
    axis : {0 or 'index', 1 or 'columns'}, default 0
        0 or 'index' for row-wise, 1 or 'columns' for column-wise
    level : int or level name, default None
        If the axis is a MultiIndex (hierarchical), count along a
        particular level, collapsing into a DataFrame
    numeric_only : boolean, default False
        Include only float, int, boolean data
    
    Returns
    -------
    count : Series (or DataFrame if level specified)



In [12]:
clientDataFileList = []
for i in range(1,5):
    clientDataFileList.append('user-report-export('+str(i)+').json')

In [13]:
clientDataFileList

['user-report-export(1).json',
 'user-report-export(2).json',
 'user-report-export(3).json',
 'user-report-export(4).json']

In [4]:
'clientDataFileList'+'i'

'clientDataFileListi'

In [4]:
# getting names of columns
visitorFile = visitors_list[1:]
with open(visitorFile, encoding='utf8') as f:
    rawData = json.load(f)
visitorDataFrameColumns = compoundColumnsNamesList(rawData)

# constructing dataFrame clientDataFrame with a visitor's data
visitorsData = pd.empty
for visitorFile in visitors_list:
    clientID = visitorFile.partition('.')[0]
    with open(visitorFile, encoding='utf8') as f:
        rawData = json.load(f)
    visitorDataFrame = json2dataFrame(clientID, 
                                 rawData, 
                                 'dates', 
                                 ['sessions', 'activities', 'details'], 
                                 visitorDataFrameColumns[:9]+clientDataFrameColumns[10:12])
# combining seperate visitors'data into single dataFrame visitorsData
    visitorsData = pd.concat([visitorsData,visitorDataFrame])

NameError: name 'visitors_list' is not defined

In [5]:
clientFile = '1155947801.1491589195.json'

In [6]:
clientID = clientFile.partition('.')[0]

In [7]:
with open(clientFile, encoding='utf8') as f:
    rawData = json.load(f)

In [11]:
clientDataFrameColumns = compoundColumnsNamesList(rawData, 'dates')

In [12]:
clientDataFrame = json2dataFrame(clientID, 
                                 rawData, 
                                 'dates', 
                                 ['sessions', 'activities', 'details'], 
                                 clientDataFrameColumns[:9]+clientDataFrameColumns[10:12])

In [13]:
clientDataFrame

,URL страницы,Заголовок страницы,date,hasGoal,hasRevenue,sessionCount,sessions.activitySummary - PAGEVIEW,sessions.channel,sessions.deviceCategory,sessions.duration,sessions.activities.pageTitle,sessions.activities.time,sessions.activities.type,repeatActivityTimes,Client ID
0,[/ru/press-center/news/03111-Intersoft-Lab-pre...,[Intersoft Lab предлагает развивать SME-бизнес...,"июн 29, 2018",False,False,1,1,Social,desktop,00:00,Intersoft Lab предлагает развивать SME-бизнес ...,9:02 AM,PAGEVIEW,[],1155947801
1,[/ru/press-center/news/03111-Intersoft-Lab-pre...,[Intersoft Lab предлагает развивать SME-бизнес...,"июн 28, 2018",False,False,2,1,Social,desktop,00:00,Intersoft Lab предлагает развивать SME-бизнес ...,9:13 AM,PAGEVIEW,[],1155947801
2,[/ru/],[Intersoft Lab],"июн 28, 2018",False,False,2,1,Social,desktop,00:00,Intersoft Lab,7:42 AM,PAGEVIEW,[],1155947801
3,[/ru/press-center/publications/03109-Aktualnye...,[Актуальные тенденции IT в финансовой сфере | ...,"июн 27, 2018",False,False,2,1,Social,desktop,00:00,Актуальные тенденции IT в финансовой сфере | П...,11:59 AM,PAGEVIEW,[],1155947801
4,[/ru/],[Intersoft Lab],"июн 27, 2018",False,False,2,13,Social,desktop,22:30,Intersoft Lab,9:15 AM,PAGEVIEW,[],1155947801
5,[/ru/press-center/publications/03110-Demokrati...,[Демократизация CPM: всем сестрам по серьгам |...,"июн 27, 2018",False,False,2,13,Social,desktop,22:30,Демократизация CPM: всем сестрам по серьгам | ...,9:14 AM,PAGEVIEW,[],1155947801
6,[/ru/],[Intersoft Lab],"июн 27, 2018",False,False,2,13,Social,desktop,22:30,Intersoft Lab,9:14 AM,PAGEVIEW,[],1155947801
7,[/ru/press-center/publications/03109-Aktualnye...,[Актуальные тенденции IT в финансовой сфере | ...,"июн 27, 2018",False,False,2,13,Social,desktop,22:30,Актуальные тенденции IT в финансовой сфере | П...,9:11 AM,PAGEVIEW,[],1155947801
8,[/ru/],[Intersoft Lab],"июн 27, 2018",False,False,2,13,Social,desktop,22:30,Intersoft Lab,9:09 AM,PAGEVIEW,[],1155947801
9,[/ru/press-center/publications/03109-Aktualnye...,[Актуальные тенденции IT в финансовой сфере | ...,"июн 27, 2018",False,False,2,13,Social,desktop,22:30,Актуальные тенденции IT в финансовой сфере | П...,9:06 AM,PAGEVIEW,[],1155947801


In [20]:
clientDataFrame.to_excel('1155947801.1491589195.xls')